In [1]:
%load_ext autoreload
%autoreload 2

In [19]:
import torch
import os
from utils.common import (
    m2f_dataset_collate,
    m2f_extract_pred_maps_and_masks,
    BG_VALUE_0,
    set_seed,
    pixel_mean_std,
    CADIS_PIXEL_MEAN,
    CADIS_PIXEL_STD,
    FULL_MERGE_PIXEL_MEAN,
    FULL_MERGE_PIXEL_STD,
)
from utils.dataset_utils import (
    get_cadisv2_dataset,
    get_cataract1k_dataset,
    ZEISS_CATEGORIES,
)
from utils.medical_datasets import Mask2FormerDataset
from transformers import (
    Mask2FormerForUniversalSegmentation,
    SwinModel,
    SwinConfig,
    Mask2FormerConfig,
    AutoImageProcessor,
    Mask2FormerImageProcessor
)
from torch.utils.data import DataLoader
import evaluate
import torch.optim as optim
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import numpy as np
from dotenv import load_dotenv
import wandb
from copy import deepcopy
import shutil
from utils.wandb_utils import log_table_of_images

In [3]:
set_seed(42) # seed everything

Random seed set as 42


In [4]:
NUM_CLASSES = len(ZEISS_CATEGORIES) - 3 + 1 # Remove class incremental and add background !!!!
SWIN_BACKBONE = "microsoft/swin-tiny-patch4-window7-224"#"microsoft/swin-large-patch4-window12-384"

# Download pretrained swin model
swin_model = SwinModel.from_pretrained(
    SWIN_BACKBONE, out_features=["stage1", "stage2", "stage3", "stage4"]
)
swin_config = SwinConfig.from_pretrained(
    SWIN_BACKBONE, out_features=["stage1", "stage2", "stage3", "stage4"]
)

# Create Mask2Former configuration based on Swin's configuration
mask2former_config = Mask2FormerConfig(
    backbone_config=swin_config, num_labels=NUM_CLASSES #, ignore_value=BG_VALUE
)

# Create the Mask2Former model with this configuration
model = Mask2FormerForUniversalSegmentation(mask2former_config)

# Reuse pretrained parameters
for swin_param, m2f_param in zip(
    swin_model.named_parameters(),
    model.model.pixel_level_module.encoder.named_parameters(),
):
    m2f_param_name = f"model.pixel_level_module.encoder.{m2f_param[0]}"

    if swin_param[0] == m2f_param[0]:
        model.state_dict()[m2f_param_name].copy_(swin_param[1])
        continue

    print(f"Not Matched: {m2f_param[0]} != {swin_param[0]}")

/notebooks/venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Not Matched: hidden_states_norms.stage1.weight != layernorm.weight
Not Matched: hidden_states_norms.stage1.bias != layernorm.bias


In [5]:
# Helper function to load datasets
def load_dataset(dataset_getter, data_path, domain_incremental):
    return dataset_getter(data_path, domain_incremental=domain_incremental)


# Helper function to create dataloaders for a dataset
def create_dataloaders(
    dataset, batch_size, shuffle, num_workers, drop_last, pin_memory, collate_fn
):
    return {
        "train": DataLoader(
            dataset["train"],
            batch_size=batch_size,
            shuffle=shuffle,
            num_workers=num_workers,
            drop_last=drop_last,
            pin_memory=pin_memory,
            collate_fn=collate_fn,
        ),
        "val": DataLoader(
            dataset["val"],
            batch_size=batch_size,
            shuffle=shuffle,
            num_workers=num_workers,
            drop_last=drop_last,
            pin_memory=pin_memory,
            collate_fn=collate_fn,
        ),
        "test": DataLoader(
            dataset["test"],
            batch_size=batch_size,
            shuffle=shuffle,
            num_workers=num_workers,
            drop_last=drop_last,
            pin_memory=pin_memory,
            collate_fn=collate_fn,
        ),
    }


# Load datasets
datasets = {
    "A": load_dataset(get_cadisv2_dataset, "../../storage/data/CaDISv2", True),
    "B": load_dataset(get_cataract1k_dataset, "../../storage/data/cataract-1k", True),
}

pixel_mean_A,pixel_std_A=pixel_mean_std(datasets["A"][0])
print("dataset A pixel mean:",pixel_mean_A,"pixel_std:",pixel_std_A)
#pixel_mean_A = CADIS_PIXEL_MEAN
#pixel_std_A = CADIS_PIXEL_STD

# This time define the B train dataset such that it replays all the training samples from A
new_train = torch.utils.data.ConcatDataset([datasets["A"][0], datasets["B"][0]])

pixel_mean_B,pixel_std_B=pixel_mean_std(new_train)
print("dataset B pixel mean:",pixel_mean_B,"pixel_std:",pixel_std_B)
#pixel_mean_B = FULL_MERGE_PIXEL_MEAN
#pixel_std_B = FULL_MERGE_PIXEL_STD

datasets["B"] = (new_train, datasets["B"][1], datasets["B"][2])

# Define preprocessor
swin_processor = AutoImageProcessor.from_pretrained(SWIN_BACKBONE)
m2f_preprocessor_A = Mask2FormerImageProcessor(
    reduce_labels=False,
    ignore_index=255,
    do_resize=False,
    do_rescale=False,
    do_normalize=True,
    image_std=pixel_std_A,
    image_mean=pixel_mean_A,
)

m2f_preprocessor_B = Mask2FormerImageProcessor(
    reduce_labels=False,
    ignore_index=255,
    do_resize=False,
    do_rescale=False,
    do_normalize=True,
    image_std=pixel_std_B,
    image_mean=pixel_mean_B,
)

# Create Mask2Former Datasets

m2f_datasets = {
    "A": {
        "train": Mask2FormerDataset(datasets["A"][0], m2f_preprocessor_A),
        "val": Mask2FormerDataset(datasets["A"][1], m2f_preprocessor_A),
        "test": Mask2FormerDataset(datasets["A"][2], m2f_preprocessor_A),
    },
    "B": {
        "train": Mask2FormerDataset(datasets["B"][0], m2f_preprocessor_B),
        "val": Mask2FormerDataset(datasets["B"][1], m2f_preprocessor_B),
        "test": Mask2FormerDataset(datasets["B"][2], m2f_preprocessor_B),
    },
}


# DataLoader parameters
N_WORKERS = 4
BATCH_SIZE = 16
SHUFFLE = True
DROP_LAST = True

dataloader_params = {
    "batch_size": BATCH_SIZE,
    "shuffle": SHUFFLE,
    "num_workers": N_WORKERS,
    "drop_last": DROP_LAST,
    "pin_memory": True,
    "collate_fn": m2f_dataset_collate,
}

# Create DataLoaders
dataloaders = {
    key: create_dataloaders(m2f_datasets[key], **dataloader_params)
    for key in m2f_datasets
}

print(dataloaders)

dataset A pixel mean: [0.57365126 0.34606295 0.19539679] pixel_std: [0.15933991 0.15584118 0.10485045]


/notebooks/venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


dataset B pixel mean: [0.48640466 0.32646684 0.20089334] pixel_std: [0.24972845 0.19719756 0.15521158]
{'A': {'train': <torch.utils.data.dataloader.DataLoader object at 0x7f999b1b6660>, 'val': <torch.utils.data.dataloader.DataLoader object at 0x7f9996122f30>, 'test': <torch.utils.data.dataloader.DataLoader object at 0x7f9996122e10>}, 'B': {'train': <torch.utils.data.dataloader.DataLoader object at 0x7f9996123110>, 'val': <torch.utils.data.dataloader.DataLoader object at 0x7f9996122f90>, 'test': <torch.utils.data.dataloader.DataLoader object at 0x7f9996123020>}}


In [6]:
m2f_preprocessor_A.reduce_labels, m2f_preprocessor_A.ignore_index

(False, 255)

In [21]:
# Check if CUDA is available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {device}")

BG_VALUE_255=255
base_run_name="M2F-Swin-Tiny-Train_Cadis"
new_run_name="M2F-Swin-Tiny-Replay-All"
project_name = "M2F_latest"
user_or_team = "continual-learning-tum"

Using device: cuda


In [8]:
# Tensorboard setup
out_dir="outputs/"
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
if not os.path.exists(out_dir+"runs"):
    os.makedirs(out_dir+"runs")
%load_ext tensorboard
%tensorboard --logdir outputs/runs

In [9]:
!CUDA_LAUNCH_BLOCKING=1

In [10]:
# Tensorboard logging
writer = SummaryWriter(log_dir=out_dir + "runs")

# Model checkpointing
model_dir = out_dir + "models/"
if not os.path.exists(model_dir):
    print("Store weights in: ", model_dir)
    os.makedirs(model_dir)

best_model_dir = model_dir + f"{base_run_name}/best_model/"
if not os.path.exists(best_model_dir):
    print("Store best model weights in: ", best_model_dir)
    os.makedirs(best_model_dir)
final_model_dir = model_dir + f"{base_run_name}/final_model/"
if not os.path.exists(final_model_dir):
    print("Store final model weights in: ", final_model_dir)
    os.makedirs(final_model_dir)

In [11]:
# WandB for team usage !!!!

wandb.login() # use this one if a different person is going to run the notebook
#wandb.login(relogin=False) # if the same person in the last run is going to run the notebook again


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ge85ket (continual-learning-tum). Use `wandb login --relogin` to force relogin


True

# First train on dataset A

In [12]:
# Training
NUM_EPOCHS = 200
LEARNING_RATE = 1e-4
LR_MULTIPLIER = 0.1
BACKBONE_LR = LEARNING_RATE * LR_MULTIPLIER
WEIGHT_DECAY = 0.05
PATIENCE=15
metric = evaluate.load("mean_iou") # mIoU will be used to pick the best performing model using val set
encoder_params = [
    param
    for name, param in model.named_parameters()
    if name.startswith("model.pixel_level_module.encoder")
]
decoder_params = [
    param
    for name, param in model.named_parameters()
    if name.startswith("model.pixel_level_module.decoder")
]
transformer_params = [
    param
    for name, param in model.named_parameters()
    if name.startswith("model.transformer_module")
]
class_prediction_params=[
    param
    for name, param in model.named_parameters() 
    if not name.startswith("model.pixel_level_module.encoder") and not name.startswith("model.transformer_module") and not name.startswith("model.pixel_level_module.decoder")
]
optimizer = optim.AdamW(
    [
        {"params": encoder_params, "lr": BACKBONE_LR},
        {"params": decoder_params},
        {"params": transformer_params},
        {"params": class_prediction_params}
    ],
    lr=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
)

scheduler = optim.lr_scheduler.PolynomialLR(
    optimizer, total_iters=NUM_EPOCHS, power=0.9
)

In [13]:
wandb.init(
    project=project_name,
    config={
        "learning_rate": LEARNING_RATE,
        "learning_rate_multiplier": LR_MULTIPLIER,
        "backbone_learning_rate": BACKBONE_LR,
        "learning_rate_scheduler": scheduler.__class__.__name__,
        "optimizer": optimizer.__class__.__name__,
        "backbone": SWIN_BACKBONE,
        "m2f_preprocessor": m2f_preprocessor_A.__dict__,
        "m2f_model_config": model.config
    },
    name=base_run_name,
    notes="M2F with tiny Swin backbone pretrained on ImageNet-1K. \
        Scenario: Train on A, Test on A"
)
print("wandb run id:",wandb.run.id)

wandb run id: 5xlx4bjp


In [14]:
# Save the preprocessor
m2f_preprocessor_A.save_pretrained(model_dir + base_run_name)

['outputs/models/M2F-Swin-Tiny-Train_Cadis/preprocessor_config.json']

In [15]:
# To avoid making stupid errors
CURR_TASK = "A"

# For storing the model
best_val_metric = -np.inf
best_model_weights=None # best model weights are stored here

# Move model to device
model.to(device)
counter=0
for epoch in range(NUM_EPOCHS):
    
    model.train()
    train_running_loss = 0.0
    val_running_loss = 0.0

    # Set up tqdm for the training loop
    train_loader = tqdm(
        dataloaders[CURR_TASK]["train"], desc=f"Epoch {epoch + 1}/{NUM_EPOCHS} Training"
    )

    for batch in train_loader:
        # Move everything to the device
        batch["pixel_values"] = batch["pixel_values"].to(device)
        batch["pixel_mask"] = batch["pixel_mask"].to(device)
        batch["mask_labels"] = [entry.to(device) for entry in batch["mask_labels"]]
        batch["class_labels"] = [entry.to(device) for entry in batch["class_labels"]]
       
        # Compute output and loss
        outputs = model(**batch)

        loss = outputs.loss
        # Compute gradient and perform step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Record losses
        current_loss = loss.item() * batch["pixel_values"].size(0)
        train_running_loss += current_loss
        train_loader.set_postfix(loss=f"{current_loss:.4f}")

        # Extract and compute metrics
        pred_maps, masks = m2f_extract_pred_maps_and_masks(
            batch, outputs, m2f_preprocessor_A
        )
        metric.add_batch(references=masks, predictions=pred_maps)
        
    # After compute the batches that were added are deleted
    temp_metrics = metric.compute(
        num_labels=NUM_CLASSES, ignore_index=BG_VALUE_255, reduce_labels=False
    )
    mean_train_iou=temp_metrics["mean_iou"]
        
    # Validation phase
    model.eval()
    val_loader = tqdm(
        dataloaders[CURR_TASK]["val"], desc=f"Epoch {epoch + 1}/{NUM_EPOCHS} Validation"
    )
    with torch.no_grad():
        for batch in val_loader:
            # Move everything to the device
            batch["pixel_values"] = batch["pixel_values"].to(device)
            batch["pixel_mask"] = batch["pixel_mask"].to(device)
            batch["mask_labels"] = [entry.to(device) for entry in batch["mask_labels"]]
            batch["class_labels"] = [
                entry.to(device) for entry in batch["class_labels"]
            ]
            # Compute output and loss
            outputs = model(**batch)

            loss = outputs.loss
            # Record losses
            current_loss = loss.item() * batch["pixel_values"].size(0)
            val_running_loss += current_loss
            val_loader.set_postfix(loss=f"{current_loss:.4f}")

            # Extract and compute metrics
            pred_maps, masks = m2f_extract_pred_maps_and_masks(
                batch, outputs, m2f_preprocessor_A
            )
            metric.add_batch(references=masks, predictions=pred_maps)
            

    # After compute the batches that were added are deleted
    mean_val_iou = metric.compute(
        num_labels=NUM_CLASSES, ignore_index=BG_VALUE_255, reduce_labels=False
    )["mean_iou"]
    
    epoch_train_loss = train_running_loss / len(dataloaders[CURR_TASK]["train"].dataset)
    epoch_val_loss = val_running_loss / len(dataloaders[CURR_TASK]["val"].dataset)

    writer.add_scalar(f"Loss/train_{base_run_name}_{CURR_TASK}", epoch_train_loss, epoch + 1)
    writer.add_scalar(f"Loss/val_{base_run_name}_{CURR_TASK}", epoch_val_loss, epoch + 1)
    writer.add_scalar(f"mIoU/train_{base_run_name}_{CURR_TASK}", mean_train_iou, epoch + 1)
    writer.add_scalar(f"mIoU/val_{base_run_name}_{CURR_TASK}", mean_val_iou, epoch + 1)

    wandb.log({
        f"Loss/train_{CURR_TASK}": epoch_train_loss,
        f"Loss/val_{CURR_TASK}": epoch_val_loss,
        f"mIoU/train_{CURR_TASK}": mean_train_iou,
        f"mIoU/val_{CURR_TASK}": mean_val_iou
    })


    tqdm.write(
        f"Epoch {epoch + 1}/{NUM_EPOCHS}, Train Loss: {epoch_train_loss:.4f}, Train mIoU: {mean_train_iou:.4f}, Validation Loss: {epoch_val_loss:.4f}, Validation mIoU: {mean_val_iou:.4f}"
    )
    
    if mean_val_iou > best_val_metric:
        best_val_metric = mean_val_iou
        #model.save_pretrained(f"{best_model_dir}{CURR_TASK}/")    
        best_model_weights = deepcopy(model.state_dict())
        counter=0
        
    else:
        counter+=1
        if counter == PATIENCE:
            print("Early stopping at epoch",epoch)
            break

os.makedirs(f"{best_model_dir}{CURR_TASK}/",exist_ok=True)
artifact = wandb.Artifact(f"best_model_{base_run_name}", type="model")
artifact.add_file(f"{best_model_dir}{CURR_TASK}/best_model_{base_run_name}.pth", torch.save(best_model_weights, f"{best_model_dir}{CURR_TASK}/best_model_{base_run_name}.pth"))
wandb.run.log_artifact(artifact)
if os.path.exists(model_dir + f"{base_run_name}"):
    shutil.rmtree(model_dir + f"{base_run_name}")


Epoch 1/200 Validation: 100%|██████████| 33/33 [01:15<00:00,  2.29s/it, loss=309.0137]


Epoch 1/200, Train Loss: 55.1861, Train mIoU: 0.1385, Validation Loss: 21.8432, Validation mIoU: 0.3302


Epoch 2/200 Validation: 100%|██████████| 33/33 [01:15<00:00,  2.29s/it, loss=300.2748]


Epoch 2/200, Train Loss: 16.5509, Train mIoU: 0.5099, Validation Loss: 15.4996, Validation mIoU: 0.5708


Epoch 3/200 Validation: 100%|██████████| 33/33 [01:05<00:00,  1.97s/it, loss=243.3515]


...

Epoch 43/200, Train Loss: 4.9988, Train mIoU: 0.9174, Validation Loss: 14.0678, Validation mIoU: 0.7667


Epoch 44/200 Validation: 100%|██████████| 33/33 [01:08<00:00,  2.08s/it, loss=309.3239]


Epoch 44/200, Train Loss: 4.7892, Train mIoU: 0.9172, Validation Loss: 14.5752, Validation mIoU: 0.7343


Epoch 45/200 Validation: 100%|██████████| 33/33 [01:05<00:00,  1.98s/it, loss=284.9697]


Epoch 45/200, Train Loss: 4.8249, Train mIoU: 0.9206, Validation Loss: 14.1229, Validation mIoU: 0.7458
Early stopping at epoch 44


## Test results on A

In [29]:

wandb.init(project=project_name, resume="allow", id="5xlx4bjp" )

In [30]:
# Load best model and evaluate on test
#model = Mask2FormerForUniversalSegmentation.from_pretrained(f"{best_model_dir}{CURR_TASK}/").to(device)

# Load pretrained on Cadis from naive forgetting 
#model = Mask2FormerForUniversalSegmentation.from_pretrained(f"/notebooks/continual-learning/outputs/models/{base_run_name}/best_model/A").to(device)

# Construct the artifact path
artifact_path = f"{user_or_team}/{project_name}/best_model_{base_run_name}:latest"

# Load from W&B
api = wandb.Api()
artifact=api.artifact(artifact_path)
model_dir=artifact.download()
model_state_dict_path = os.path.join(model_dir, f"best_model_{base_run_name}.pth" )
model_state_dict = torch.load(model_state_dict_path)
model = Mask2FormerForUniversalSegmentation(mask2former_config)
model.load_state_dict(model_state_dict)
model.to(device)

wandb: Downloading large artifact best_model_M2F-Swin-Tiny-Train_Cadis:latest, 181.31MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.6


Mask2FormerForUniversalSegmentation(
  (model): Mask2FormerModel(
    (pixel_level_module): Mask2FormerPixelLevelModule(
      (encoder): SwinBackbone(
        (embeddings): SwinEmbeddings(
          (patch_embeddings): SwinPatchEmbeddings(
            (projection): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
          )
          (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.0, inplace=False)
        )
        (encoder): SwinEncoder(
          (layers): ModuleList(
            (0): SwinStage(
              (blocks): ModuleList(
                (0-1): 2 x SwinLayer(
                  (layernorm_before): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
                  (attention): SwinAttention(
                    (self): SwinSelfAttention(
                      (query): Linear(in_features=96, out_features=96, bias=True)
                      (key): Linear(in_features=96, out_features=96, bias=True)
                      (value

In [27]:
model.eval()
test_running_loss = 0
CURR_TASK="A"
test_loader = tqdm(dataloaders[CURR_TASK]["test"], desc="Test loop")

BATCH_INDEX = 0
table = wandb.Table(columns=["ID", "Image"])
with torch.no_grad():
    for batch in test_loader:
        # Move everything to the device
        batch["pixel_values"] = batch["pixel_values"].to(device)
        batch["pixel_mask"] = batch["pixel_mask"].to(device)
        batch["mask_labels"] = [entry.to(device) for entry in batch["mask_labels"]]
        batch["class_labels"] = [entry.to(device) for entry in batch["class_labels"]]
        # Compute output and loss
        outputs = model(**batch)

        loss = outputs.loss
        # Record losses
        current_loss = loss.item() * batch["pixel_values"].size(0)
        test_running_loss += current_loss
        test_loader.set_postfix(loss=f"{current_loss:.4f}")

        # Extract and compute metrics
        pred_maps, masks = m2f_extract_pred_maps_and_masks(
            batch, outputs, m2f_preprocessor_A
        )
        metric.add_batch(references=masks, predictions=pred_maps)
        if BATCH_INDEX <5:
            # Visualize
            log_table_of_images(
                table, # common table for all batches
                batch["pixel_values"],
                pixel_mean_A, # remove normalization
                pixel_std_A, # remove normalization
                pred_maps,
                masks,
                BATCH_INDEX, # correct indexing in table
            )
            BATCH_INDEX += 1
# Log table
wandb.log({f"{CURR_TASK}_TEST": table})

# After compute the batches that were added are deleted
test_metrics_A = metric.compute(
    num_labels=NUM_CLASSES, ignore_index=BG_VALUE_255, reduce_labels=False
)
mean_test_iou = test_metrics_A["mean_iou"]
final_test_loss = test_running_loss / len(dataloaders[CURR_TASK]["test"].dataset)
wandb.log({
    f"Loss/test_{CURR_TASK}": final_test_loss,
    f"mIoU/test_{CURR_TASK}": mean_test_iou
})
print(f"Test Loss: {final_test_loss:.4f}, Test mIoU: {mean_test_iou:.4f}")

Test loop: 100%|██████████| 36/36 [02:17<00:00,  3.83s/it, loss=197.0663]


Test Loss: 16.2424, Test mIoU: 0.7469


## Test results on B

In [31]:
model.eval()
test_running_loss = 0
CURR_TASK="B"
test_loader = tqdm(dataloaders[CURR_TASK]["test"], desc="Test loop")

BATCH_INDEX = 0
table = wandb.Table(columns=["ID", "Image"])
with torch.no_grad():
    for batch in test_loader:
        # Move everything to the device
        batch["pixel_values"] = batch["pixel_values"].to(device)
        batch["pixel_mask"] = batch["pixel_mask"].to(device)
        batch["mask_labels"] = [entry.to(device) for entry in batch["mask_labels"]]
        batch["class_labels"] = [entry.to(device) for entry in batch["class_labels"]]
        # Compute output and loss
        outputs = model(**batch)

        loss = outputs.loss
        # Record losses
        current_loss = loss.item() * batch["pixel_values"].size(0)
        test_running_loss += current_loss
        test_loader.set_postfix(loss=f"{current_loss:.4f}")

        # Extract and compute metrics
        pred_maps, masks = m2f_extract_pred_maps_and_masks(
            batch, outputs, m2f_preprocessor_B
        )
        metric.add_batch(references=masks, predictions=pred_maps)
        if BATCH_INDEX <5:
            # Visualize
            log_table_of_images(
                table, # common table for all batches
                batch["pixel_values"],
                pixel_mean_B, # remove normalization
                pixel_std_B, # remove normalization
                pred_maps,
                masks,
                BATCH_INDEX, # correct indexing in table
            )
            BATCH_INDEX += 1

# Log table
wandb.log({f"{CURR_TASK}_TEST": table})

# After compute the batches that were added are deleted
test_metrics_B_before = metric.compute(
    num_labels=NUM_CLASSES, ignore_index=BG_VALUE_255, reduce_labels=False
)
mean_test_iou = test_metrics_B_before["mean_iou"]
final_test_loss = test_running_loss / len(dataloaders[CURR_TASK]["test"].dataset)
wandb.log({
    f"Loss/test_{CURR_TASK}": final_test_loss,
    f"mIoU/test_{CURR_TASK}": mean_test_iou
})
print(f"Test Loss: {final_test_loss:.4f}, Test mIoU: {mean_test_iou:.4f}")


Test loop: 100%|██████████| 14/14 [00:36<00:00,  2.58s/it, loss=802.0696]


Test Loss: 47.3692, Test mIoU: 0.2325


In [32]:
wandb.finish()

Loss/test_A,16.19772
Loss/test_B,47.76248
Loss/train_A,4.82494
Loss/val_A,14.12289
mIoU/test_A,0.74521
mIoU/test_B,0.23032
mIoU/train_A,0.92061
mIoU/val_A,0.74581


# Now train on B and forget A

In [ ]:
# Training
NUM_EPOCHS = 200
LEARNING_RATE = 1e-4
LR_MULTIPLIER = 0.1
BACKBONE_LR = LEARNING_RATE * LR_MULTIPLIER
WEIGHT_DECAY = 0.05
PATIENCE=15
metric = evaluate.load("mean_iou")
encoder_params = [
    param
    for name, param in model.named_parameters()
    if name.startswith("model.pixel_level_module.encoder")
]
decoder_params = [
    param
    for name, param in model.named_parameters()
    if name.startswith("model.pixel_level_module.decoder")
]
transformer_params = [
    param
    for name, param in model.named_parameters()
    if name.startswith("model.transformer_module")
]
class_prediction_params=[
    param
    for name, param in model.named_parameters() 
    if not name.startswith("model.pixel_level_module.encoder") and not name.startswith("model.transformer_module") and not name.startswith("model.pixel_level_module.decoder")
]
optimizer = optim.AdamW(
    [
        {"params": encoder_params, "lr": BACKBONE_LR},
        {"params": decoder_params},
        {"params": transformer_params},
        {"params": class_prediction_params}
    ],
    lr=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
)

scheduler = optim.lr_scheduler.PolynomialLR(
    optimizer, total_iters=NUM_EPOCHS, power=0.9
)

In [ ]:
# WandB for team usage !!!!

wandb.login() # use this one if a different person is going to run the notebook
#wandb.login(relogin=False) # if the same person in the last run is going to run the notebook again

In [ ]:
wandb.init(
    project=project_name,
    config={
        "learning_rate": LEARNING_RATE,
        "learning_rate_multiplier": LR_MULTIPLIER,
        "backbone_learning_rate": BACKBONE_LR,
        "learning_rate_scheduler": scheduler.__class__.__name__,
        "optimizer": optimizer.__class__.__name__,
        "backbone": SWIN_BACKBONE,
        "m2f_preprocessor": m2f_preprocessor_B.__dict__,
        "m2f_model_config": model.config
    },
    name=new_run_name,
    notes="M2F with tiny Swin backbone pretrained on ImageNet-1K. \
        Scenario: Pretrained on A, Train on A + B naive finetuning (replay all), Test forgetting on A"
)

print("wandb run id:",wandb.run.id)

# Tensorboard logging
writer = SummaryWriter(log_dir=out_dir + "runs")

# Model checkpointing
model_dir = out_dir + "models/"
if not os.path.exists(model_dir):
    print("Store weights in: ", model_dir)
    os.makedirs(model_dir)

best_model_dir = model_dir + f"{new_run_name}/best_model/"
if not os.path.exists(best_model_dir):
    print("Store best model weights in: ", best_model_dir)
    os.makedirs(best_model_dir)
final_model_dir = model_dir + f"{new_run_name}/final_model/"
if not os.path.exists(final_model_dir):
    print("Store final model weights in: ", final_model_dir)
    os.makedirs(final_model_dir)

In [ ]:
# Save the preprocessor
m2f_preprocessor_B.save_pretrained(model_dir + new_run_name)

In [ ]:
# To avoid making stupid errors
CURR_TASK = "B"
model_path_second = f"{best_model_dir}A+{CURR_TASK}/"

# For storing the model
best_val_metric = -np.inf
best_model_weights=None # best model weights are stored here

# Move model to device
model.to(device)
counter=0
for epoch in range(NUM_EPOCHS):
    model.train()
    train_running_loss = 0.0
    val_running_loss = 0.0

    # Set up tqdm for the training loop
    train_loader = tqdm(
        dataloaders[CURR_TASK]["train"], desc=f"Epoch {epoch + 1}/{NUM_EPOCHS} Training"
    )

    for batch in train_loader:
        # Move everything to the device
        batch["pixel_values"] = batch["pixel_values"].to(device)
        batch["pixel_mask"] = batch["pixel_mask"].to(device)
        batch["mask_labels"] = [entry.to(device) for entry in batch["mask_labels"]]
        batch["class_labels"] = [entry.to(device) for entry in batch["class_labels"]]

        # Compute output and loss
        outputs = model(**batch)

        loss = outputs.loss

        # Compute gradient and perform step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Record losses
        current_loss = loss.item() * batch["pixel_values"].size(0)
        train_running_loss += current_loss
        train_loader.set_postfix(loss=f"{current_loss:.4f}")

        # Extract and compute metrics
        pred_maps, masks = m2f_extract_pred_maps_and_masks(
            batch, outputs, m2f_preprocessor_B
        )
        metric.add_batch(references=masks, predictions=pred_maps)
        

    # After compute the batches that were added are deleted
    mean_train_iou = metric.compute(
        num_labels=NUM_CLASSES, ignore_index=BG_VALUE_255, reduce_labels=False
    )["mean_iou"]

    # Validation phase
    model.eval()
    val_loader = tqdm(
        dataloaders[CURR_TASK]["val"], desc=f"Epoch {epoch + 1}/{NUM_EPOCHS} Validation"
    )
    with torch.no_grad():
        for batch in val_loader:
            # Move everything to the device
            batch["pixel_values"] = batch["pixel_values"].to(device)
            batch["pixel_mask"] = batch["pixel_mask"].to(device)
            batch["mask_labels"] = [entry.to(device) for entry in batch["mask_labels"]]
            batch["class_labels"] = [
                entry.to(device) for entry in batch["class_labels"]
            ]
            # Compute output and loss
            outputs = model(**batch)

            loss = outputs.loss
            # Record losses
            current_loss = loss.item() * batch["pixel_values"].size(0)
            val_running_loss += current_loss
            val_loader.set_postfix(loss=f"{current_loss:.4f}")

            # Extract and compute metrics
            pred_maps, masks = m2f_extract_pred_maps_and_masks(
                batch, outputs, m2f_preprocessor_B
            )
            metric.add_batch(references=masks, predictions=pred_maps)
            

    # After compute the batches that were added are deleted
    mean_val_iou = metric.compute(
        num_labels=NUM_CLASSES, ignore_index=BG_VALUE_255, reduce_labels=False
    )["mean_iou"]

    epoch_train_loss = train_running_loss / len(dataloaders[CURR_TASK]["train"].dataset)
    epoch_val_loss = val_running_loss / len(dataloaders[CURR_TASK]["val"].dataset)

    writer.add_scalar(f"Loss/train_{new_run_name}_{CURR_TASK}", epoch_train_loss, epoch + 1)
    writer.add_scalar(f"Loss/val_{new_run_name}_{CURR_TASK}", epoch_val_loss, epoch + 1)
    writer.add_scalar(f"mIoU/train_{new_run_name}_{CURR_TASK}", mean_train_iou, epoch + 1)
    writer.add_scalar(f"mIoU/val_{new_run_name}_{CURR_TASK}", mean_val_iou, epoch + 1)

    wandb.log({
        f"Loss/train_A+{CURR_TASK}": epoch_train_loss,
        f"Loss/val_A+{CURR_TASK}": epoch_val_loss,
        f"mIoU/train_A+{CURR_TASK}": mean_train_iou,
        f"mIoU/val_A+{CURR_TASK}": mean_val_iou
    })

    tqdm.write(
        f"Epoch {epoch + 1}/{NUM_EPOCHS}, Train Loss: {epoch_train_loss:.4f}, Train mIoU: {mean_train_iou:.4f}, Validation Loss: {epoch_val_loss:.4f}, Validation mIoU: {mean_val_iou:.4f}"
    )
    if mean_val_iou > best_val_metric:
        best_val_metric = mean_val_iou
        #model.save_pretrained(model_path_second)
        best_model_weights = deepcopy(model.state_dict())
        counter=0
    else:
        counter+=1
        if counter == PATIENCE:
            print("Early stopping at epoch",epoch)
            break
            

os.makedirs(f"{model_path_second}",exist_ok=True)
artifact = wandb.Artifact(f"best_model_{new_run_name}", type="model")
artifact.add_file(f"{model_path_second}/best_model_{new_run_name}.pth", torch.save(best_model_weights, f"{model_path_second}/best_model_{new_run_name}.pth"))
wandb.run.log_artifact(artifact)

if os.path.exists(model_dir + f"{new_run_name}"):
    shutil.rmtree(model_dir + f"{new_run_name}")

## Test results on B first

In [ ]:
# Load best model and evaluate on test
#model = Mask2FormerForUniversalSegmentation.from_pretrained(model_path_second).to(device)

# Construct the artifact path
artifact_path = f"{user_or_team}/{project_name}/best_model_{new_run_name}:latest"

# Load from W&B
api = wandb.Api()
artifact=api.artifact(artifact_path)
model_dir=artifact.download()
model_state_dict_path = os.path.join(model_dir, f"best_model_{new_run_name}.pth" )
model_state_dict = torch.load(model_state_dict_path)
model = Mask2FormerForUniversalSegmentation(mask2former_config)
model.load_state_dict(model_state_dict)
model.to(device)

In [ ]:
model.eval()
test_running_loss = 0
CURR_TASK="B"
test_loader = tqdm(dataloaders[CURR_TASK]["test"], desc="Test loop")
with torch.no_grad():
    for batch in test_loader:
        # Move everything to the device
        batch["pixel_values"] = batch["pixel_values"].to(device)
        batch["pixel_mask"] = batch["pixel_mask"].to(device)
        batch["mask_labels"] = [entry.to(device) for entry in batch["mask_labels"]]
        batch["class_labels"] = [entry.to(device) for entry in batch["class_labels"]]
        # Compute output and loss
        outputs = model(**batch)

        loss = outputs.loss
        # Record losses
        current_loss = loss.item() * batch["pixel_values"].size(0)
        test_running_loss += current_loss
        test_loader.set_postfix(loss=f"{current_loss:.4f}")

        # Extract and compute metrics
        pred_maps, masks = m2f_extract_pred_maps_and_masks(
            batch, outputs, m2f_preprocessor_B
        )
        metric.add_batch(references=masks, predictions=pred_maps)


# After compute the batches that were added are deleted
test_metrics_B = metric.compute(
    num_labels=NUM_CLASSES, ignore_index=BG_VALUE_255, reduce_labels=False
)
mean_test_iou = test_metrics_B["mean_iou"]
final_test_loss = test_running_loss / len(dataloaders[CURR_TASK]["test"].dataset)
wandb.log({
    f"Loss/test_{CURR_TASK}": final_test_loss,
    f"mIoU/test_{CURR_TASK}": mean_test_iou
})
print(f"Test Loss: {final_test_loss:.4f}, Test mIoU: {mean_test_iou:.4f}")

## Test results on A after training on B

In [ ]:
# To avoid making stupid errors
CURR_TASK = "A"

model.eval()
test_running_loss = 0
test_loader = tqdm(dataloaders[CURR_TASK]["test"], desc="Test loop")
with torch.no_grad():
    for batch in test_loader:
        # Move everything to the device
        batch["pixel_values"] = batch["pixel_values"].to(device)
        batch["pixel_mask"] = batch["pixel_mask"].to(device)
        batch["mask_labels"] = [entry.to(device) for entry in batch["mask_labels"]]
        batch["class_labels"] = [entry.to(device) for entry in batch["class_labels"]]
        # Compute output and loss
        outputs = model(**batch)

        loss = outputs.loss
        # Record losses
        current_loss = loss.item() * batch["pixel_values"].size(0)
        test_running_loss += current_loss
        test_loader.set_postfix(loss=f"{current_loss:.4f}")

        # Extract and compute metrics
        pred_maps, masks = m2f_extract_pred_maps_and_masks(
            batch, outputs, m2f_preprocessor_A
        )
        metric.add_batch(references=masks, predictions=pred_maps)
        
# After compute the batches that were added are deleted
test_metrics_forgetting_A = metric.compute(
    num_labels=NUM_CLASSES, ignore_index=BG_VALUE_255, reduce_labels=False
)
mean_test_iou = test_metrics_forgetting_A["mean_iou"]
final_test_loss = test_running_loss / len(dataloaders[CURR_TASK]["test"].dataset)
wandb.log({
    f"Loss/test_replay_all_{CURR_TASK}": final_test_loss,
    f"mIoU/test_replay_all_{CURR_TASK}": mean_test_iou
})
print(f"Test Loss: {final_test_loss:.4f}, Test mIoU: {mean_test_iou:.4f}")


In [ ]:
# Collect overall mIoU
mIoU_A_before = test_metrics_A["mean_iou"]
mIoU_B_before=test_metrics_B_before["mean_iou"]
mIoU_forgetting_A = test_metrics_forgetting_A["mean_iou"]
mIoU_B = test_metrics_B["mean_iou"]

# Collect per category mIoU
per_category_mIoU_A_before = np.array(test_metrics_A["per_category_iou"])
per_category_mIoU_forgetting_A = np.array(test_metrics_forgetting_A["per_category_iou"])
per_category_mIoU_B = np.array(test_metrics_B["per_category_iou"])
per_category_mIoU_B_before=np.array(test_metrics_B_before["per_category_iou"])

# Average learning accuracies (mIoUs)
avg_learning_acc = (mIoU_A_before + mIoU_B) / 2
per_category_avg_learning_acc = (per_category_mIoU_A_before + per_category_mIoU_B) / 2

# Forgetting
total_forgetting = mIoU_A_before - mIoU_forgetting_A
per_category_forgetting = (per_category_mIoU_A_before - per_category_mIoU_forgetting_A)

# Export evaluation metrics to WandB
wandb.log({
    "eval/avg_learning_acc": avg_learning_acc,
    "eval/total_forgetting": total_forgetting,
})

columns=["categories","per_category_mIoU_A_before","per_category_mIoU_B_before",
         "per_category_mIoU_B", "per_category_mIoU_forgetting_A",
         "per_category_avg_learning_acc","per_category_forgetting"]
data=[]

for i in range(1,12):
    cat_id=i-1
    data.append([ZEISS_CATEGORIES[i],per_category_mIoU_A_before[cat_id],
                 per_category_mIoU_B_before[cat_id],
                 per_category_mIoU_B[cat_id],
                per_category_mIoU_forgetting_A[cat_id],per_category_avg_learning_acc[cat_id],
                per_category_forgetting[cat_id]])
    
table = wandb.Table(columns=columns, data=data)
wandb.log({"per_category_metrics_table": table})

print("**** Overall mIoU ****")
print(f"mIoU on task A before training on B: {mIoU_A_before}")
print(f"mIoU on task B before training on B: {mIoU_B_before}")
print("\n")
print(f"mIoU on task B after training on B: {mIoU_B}")
print(f"mIoU on task A after training on B: {mIoU_forgetting_A}")

print("\n**** Per category mIoU ****")
print(f"Per category mIoU on task A before training on B: {per_category_mIoU_A_before}")
print(f"Per category mIoU on task B before training on B: {per_category_mIoU_B_before}")
print("\n")
print(f"Per category mIoU on task B after training on B: {per_category_mIoU_B}")
print(f"Per category mIoU on task A after training on B: {per_category_mIoU_forgetting_A}")

print("\n**** Average learning accuracies ****")
print(f"Average learning acc.: {avg_learning_acc}")
print(f"Per category Average learning acc.: {per_category_avg_learning_acc}")

print("\n**** Forgetting ****")
print(f"Total forgetting: {total_forgetting}")
print(f"Per category forgetting: {per_category_forgetting}")
wandb.finish()

if os.path.exists("artifacts/"):
    shutil.rmtree("artifacts/")